In [1]:
# Classify PTT posts to broad by attention RNN model.
import pandas as pd
import time
import csv
import os
import types 

#Initialize for files name and path.
base_dir = 'C:/Users/User/Raw data/PTT'
totalboard_name = 'TJ-BG-AT-SX-CC-MV'
board_name = ['Tech_job','Boy-Girl','AllTogether','sex','C_Chat','movie']
board_dict_index = {'Tech_job':0,'Boy-Girl':1,'AllTogether':2,'sex':3,'C_Chat':4,'movie':5}
max_word_length = 256
min_word_length = 100
label_newnum = len(board_name)

new_content_dir = []
totalboard_dir = os.path.join(base_dir, totalboard_name)
if not os.path.exists(totalboard_dir):
    os.makedirs(totalboard_dir)
for name_ind in range(len(board_name)):
    # New data path
    new_content_dir.append(os.path.join(totalboard_dir, board_name[name_ind]+'_content'))
    if not os.path.exists(new_content_dir[name_ind]):
        os.makedirs(new_content_dir[name_ind])

from sklearn.cross_validation import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import jieba

text_content = []
text_label = []

#dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '-test.csv'))
dfs_total = pd.read_csv(os.path.join(totalboard_dir, totalboard_name + '.csv'))
len_total = len(dfs_total)

for dfs_index in range(len_total):
    read_index = int(dfs_total.iloc[dfs_index, 0])
    type_name = str(dfs_total.iloc[dfs_index, 1])
    word_length = int(dfs_total.iloc[dfs_index, 5])
    title_name = str(dfs_total.iloc[dfs_index, 9])
    this_board_name = str(dfs_total.iloc[dfs_index, 10])
    
    text = ''
    with open(os.path.join(new_content_dir[board_dict_index[this_board_name]], str(read_index) + '.csv'),
              'r', encoding = 'utf-8-sig') as file:
        csvCursor = csv.reader(file)
        for rows in csvCursor:
            for row in rows:
                # Read content and remove empty.
                text = text + row
    
    text_content.append(text)
    text_label.append(board_dict_index[this_board_name])
    file.close()

words_limit = 40000
tokenizer = Tokenizer(num_words=words_limit, 
                      filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n　，。！：；、？﹝﹞「」『』（）｛｝［］【】《》“”‘’＼｜〝〞‵′＋－＊／＝≦≧＿＠＃＄％︿＆～§◎．※ㄧ↔│○●☆★◇◆□■▽▼△▲㊣⊙⊕ˍ…﹌﹋﹎﹍﹉﹊‥–↑↓←→↖↗↙↘∥∕℅≒≡∩∪∞￣＿◤◥◣◢∵∴〒⊥∠⊿┼┴┬┤├▔─│▕┌┐└┘╭╮╰╯═╞╪╡╔╦╗╠╬╣╚╩╝╒╤╕╘╧╛╓╥╖╟╫╢╙╨╜║▓╱╲╳▁▂▄▅▆▇█▏▎▍▌▋▊▉▁▔', 
                      split=" ")
tokenizer.fit_on_texts(text_content)
vocab = tokenizer.word_index
print(len(vocab))

vocab_counts = tokenizer.word_counts
vocab_docs = tokenizer.word_docs
vocab_index = 0
for vocab_word in vocab:
    if vocab_index % 1000 == 0 or vocab_index < 20:
        print(vocab_index, vocab_word, vocab_counts[vocab_word], vocab_docs[vocab_word])
    vocab_index += 1

from keras.utils import np_utils
x_train, x_test, y_train, y_test = train_test_split(text_content, text_label,
                                                    test_size=0.2, random_state=9487)

y_train = np_utils.to_categorical(y_train, label_newnum)
y_test = np_utils.to_categorical(y_test, label_newnum)

# 将每个词用词典中的数值代替
x_train_word_ids = tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)
# 序列模式
x_train = pad_sequences(x_train_word_ids, maxlen=max_word_length)
x_test = pad_sequences(x_test_word_ids, maxlen=max_word_length)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


190190
0 的 453107 83139
1 我 184079 56942
2 是 153676 65403
3 了 126424 57126
4 有 104298 55936
5 在 94438 51777
6 也 85862 48035
7 都 73951 44112
8 就 70596 41607
9 不 64110 40699
10 很 53089 33219
11 會 51420 33428
12 他 49080 19704
13 跟 46775 29688
14 你 46472 20085
15 想 45924 32358
16 說 45526 28978
17 看 45432 27455
18 可以 45145 29301
19 她 43894 14221
1000 中間 1187 1135
2000 f 569 453
3000 實在太 343 338
4000 作畫 238 182
5000 六日 181 164
6000 力晶 144 102
7000 給些 117 117
8000 一棟 98 91
9000 免得 83 83
10000 善於 73 69
11000 隨身 64 48
12000 超時空 57 43
13000 數值 51 43
14000 罪名 46 43
15000 spec 42 37
16000 滷味 38 38
17000 有異 35 35
18000 z017da 32 32
19000 組員 30 20
20000 電影界 28 27
21000 滋潤 26 24
22000 邪神 24 22
23000 作對 22 20
24000 尤里 21 11
25000 必經 20 20
26000 抓起 19 19
27000 喇機 18 14
28000 淪落 17 17
29000 找下 16 16
30000 龍龍 15 15
31000 九淺 14 8
32000 焊接 14 8
33000 席格 13 9
34000 家長會 13 13
35000 2510p 12 6
36000 自吹 12 2
37000 大嘴巴 11 11
38000 帶你去 11 11
39000 交不 10 9
40000 街機 10 8
41000 山盟海誓 9 7
42000 點整 9 3
43000 貞潔 9 8
44

In [ ]:
vocab_index = 0
for vocab_word in vocab:
    if len(vocab_word)>2:
        print(vocab_index, vocab_word, vocab_counts[vocab_word], vocab_docs[vocab_word])
    vocab_index += 1

In [3]:
import os
import time
import numpy as np
from keras.models import Model, Input
from keras.layers import Dense, Dropout, Activation, Embedding, Concatenate, BatchNormalization, regularizers
from keras.layers import Conv1D, Flatten, Dropout, MaxPool1D, Lambda, Dot
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional
from keras.optimizers import Adam, Adadelta
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix
from keras import backend as K
import myLayers.AttentionBase

patience = 10
num_epoch = 30
batch_size = 64
save_every = 1
pretrain = 0

def build_model():
    main_input = Input(shape=(max_word_length,), dtype='int32')
    
    embedder = Embedding(40000+1, 300, input_length=max_word_length)
    embed = embedder(main_input)
    
    rnn = Bidirectional(GRU(150, dropout=0.2, recurrent_dropout=0.1, return_sequences=True, 
                            kernel_regularizer=regularizers.l2(0.01), 
                            recurrent_regularizer=regularizers.l2(0.01), 
                            bias_regularizer=regularizers.l2(0.01)))(embed)
    rnn = myLayers.AttentionBase.Attention(150*2, input_metrix=True, 
                                           kernel_regularizer=regularizers.l2(0.01))(rnn)
    rnn = Dense(60, kernel_regularizer=regularizers.l2(0.01), 
                bias_regularizer=regularizers.l2(0.01))(rnn)
    rnn = Dropout(0.5)(rnn)
    
    main_output = Dense(label_newnum, activation = 'softmax')(rnn)
    model = Model(inputs = main_input, outputs = main_output)
    #opt = 'adam'
    opt = 'Nadam'
    model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics = ['accuracy'])
    model.summary()
    return model

if pretrain == 0:
    model = build_model()
else:
    model = load_model(model_name)

model_name = 'RNN_PTT_model_fixed_' + str(time.time())
mdl_dir = os.path.join(base_dir, 'model')
if not os.path.exists(mdl_dir):
    os.makedirs(mdl_dir)
model_dir = os.path.join(mdl_dir, model_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

num_instances = len(y_train)
iter_per_epoch = int(num_instances / batch_size)
iter_pct10 = int(iter_per_epoch / 10)
if (num_instances % batch_size) > 0:
    iter_per_epoch += 1
batch_cutoff = [0]
for i in range(iter_per_epoch - 1):
    batch_cutoff.append(batch_size * (i+1))
batch_cutoff.append(num_instances)

total_start_t = time.time()
best_metrics = 0.0
best_epoch = 1
early_stop_counter = 0
for e in range(num_epoch):
    rand_idxs = np.random.permutation(num_instances)
    print('\n#######')
    print('epoch' + str(e+1))
    print('#######')
    start_t = time.time()
    for i in range(iter_per_epoch):
        X_batch = []
        Y_batch = []
        # Get random batch input.
        for n in range(batch_cutoff[i],batch_cutoff[i+1]):
            X_batch.append(x_train[rand_idxs[n]])
            Y_batch.append(y_train[rand_idxs[n]])
        loss_and_metrics = model.train_on_batch(np.asarray(X_batch),np.asarray(Y_batch))
        if i % iter_pct10 == iter_pct10 - 1:
            print('Iteration ',i+1,' (',round(i*100/iter_per_epoch,2),'%) Train Loss: ',loss_and_metrics[0],'; Train accuracy: ',loss_and_metrics[1],'\t\t',end='\n')
    
    print('Train Loss: ',loss_and_metrics[0],'; Train accuracy: ',loss_and_metrics[1])
    loss_and_metrics = model.evaluate(x_test, y_test, batch_size)
    print('Test Loss: ',loss_and_metrics[0],': Test accuracy: ',loss_and_metrics[1])

    predictions = model.predict(x_test)
    predictions = predictions.argmax(axis=-1)
    y_labels = y_test.argmax(axis=-1)
    confusions = confusion_matrix(y_labels, predictions)
    print('confusions [prediction\True]')
    print(confusions)

    if loss_and_metrics[1] >= best_metrics:
        best_metrics = loss_and_metrics[1]
        best_epoch = e
        early_stop_counter = 0
        print('Save best score!! '+str(best_metrics))
    else:
        early_stop_counter += 1
        print('Count early stop!! '+str(early_stop_counter))

    print('Elapsed time in epoch ' + str(e+1) + ': ' + str(time.time() - start_t) + ' [s]')

    if (e+1) % save_every == 0:
        model_path = os.path.join(model_dir, 'model-%d.h5' %(e+1))
        model.save(model_path)
        model_weights_path = os.path.join(model_dir, 'model_weights-%d.h5' %(e+1))
        model.save_weights(model_weights_path)
        print('Saved model %s!' %str(e+1))

    if patience != 0 and early_stop_counter >= patience:
        break

print('\n========')
print('Best model')
print('========')
print('Stop by early stopping')
print('Best score: ', best_metrics, 'Beat model: ', best_epoch)
plot_model(model, to_file=model_dir+'/model.png')
print('Elapsed time in total: ' + str(time.time() - total_start_t))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 256, 300)          12000300  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256, 300)          405900    
_________________________________________________________________
attention_1 (Attention)      (None, 300)               90000     
_________________________________________________________________
dense_1 (Dense)              (None, 60)                18060     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 366       
Total para

Iteration  756  ( 69.91 %) Train Loss:  0.0284208 ; Train accuracy:  1.0 		
Iteration  864  ( 79.91 %) Train Loss:  0.033852 ; Train accuracy:  1.0 		
Iteration  972  ( 89.91 %) Train Loss:  0.361873 ; Train accuracy:  0.96875 		
Iteration  1080  ( 99.91 %) Train Loss:  0.0496482 ; Train accuracy:  1.0 		
Train Loss:  0.0496482 ; Train accuracy:  1.0
17280/17280 [==============================] - 26s    
Test Loss:  0.238211073837 : Test accuracy:  0.96712962963
confusions [prediction\True]
[[2748   32   10    6   17    5]
 [   4 2823   15   31   21   10]
 [   1   21 2857    2    5   12]
 [   4  142   11 2690   43   26]
 [   0   15    2   16 2800   45]
 [   2   14    6    4   46 2794]]
Count early stop!! 1
Elapsed time in epoch 6: 506.0443868637085 [s]
Saved model 6!

#######
epoch7
#######
Iteration  108  ( 9.91 %) Train Loss:  0.0535448 ; Train accuracy:  0.984375 		
Iteration  216  ( 19.91 %) Train Loss:  0.0761801 ; Train accuracy:  1.0 		
Iteration  324  ( 29.91 %) Train Loss:  0.

Saved model 12!

#######
epoch13
#######
Iteration  108  ( 9.91 %) Train Loss:  0.127907 ; Train accuracy:  0.984375 		
Iteration  216  ( 19.91 %) Train Loss:  0.213258 ; Train accuracy:  0.96875 		
Iteration  324  ( 29.91 %) Train Loss:  0.0295344 ; Train accuracy:  1.0 		
Iteration  432  ( 39.91 %) Train Loss:  0.0441871 ; Train accuracy:  1.0 		
Iteration  540  ( 49.91 %) Train Loss:  0.0851306 ; Train accuracy:  0.96875 		
Iteration  648  ( 59.91 %) Train Loss:  0.0834111 ; Train accuracy:  1.0 		
Iteration  756  ( 69.91 %) Train Loss:  0.047237 ; Train accuracy:  1.0 		
Iteration  864  ( 79.91 %) Train Loss:  0.0195374 ; Train accuracy:  1.0 		
Iteration  972  ( 89.91 %) Train Loss:  0.0563899 ; Train accuracy:  1.0 		
Iteration  1080  ( 99.91 %) Train Loss:  0.0400628 ; Train accuracy:  1.0 		
Train Loss:  0.0400628 ; Train accuracy:  1.0
17280/17280 [==============================] - 26s    
Test Loss:  0.254803313398 : Test accuracy:  0.969212962963
confusions [prediction\True]